#### Mount GDrive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Unzip Data from GDrive

In [2]:
!unzip -q '/content/drive/MyDrive/csci535/Videos7439.zip'
!unzip -q '/content/drive/MyDrive/csci535/Spectrograms7439.zip'

#### Check the number of video files

In [3]:
!ls 'Videos7439' | wc -l

7439


#### Check the number of spectrogram files

In [4]:
!ls 'Spectrograms7439' | wc -l

7439


#### Assign video and spectrogram directories

In [20]:
videos_folder = 'Videos7439'
spectrograms_folder = 'Spectrograms7439'

#### Install some CV libraries

In [2]:
!pip install einops
!pip install facenet-pytorch
!pip install face_alignment
!pip install self_attention_cv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 28.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.8 MB/s eta 0:00:00


#### Import libraries

In [3]:
import cv2
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
from einops.layers.torch import Rearrange
from einops import rearrange
from facenet_pytorch import MTCNN
from self_attention_cv import TransformerEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import face_alignment
import requests

ModuleNotFoundError: No module named 'facenet_pytorch'

#### Data Pre-processing

In [ ]:
def extract_frame(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    mid_frame_index = frame_count // 2  # Index of the frame in the middle of the video
    cap.set(cv2.CAP_PROP_POS_FRAMES, mid_frame_index)
    ret, frame = cap.read()
    if ret:
        cap.release()
        return frame
    else:
        cap.release()
        return None

In [ ]:
def detect_face(frame):
    mtcnn = MTCNN()
    boxes, _ = mtcnn.detect(frame)
    if boxes is not None:
        # Assuming only one face in the frame
        box = boxes[0]
        x1, y1, x2, y2 = box
        # Crop the frame to the detected face
        cropped_frame = frame[int(y1):int(y2), int(x1):int(x2)]
        return cropped_frame
    else:
        return None

In [ ]:
# Define the preprocessing functions for video frames and spectrograms
def preprocess_image(frame):
    frame_pil = Image.fromarray(frame.astype('uint8'))
    frame_pil = frame_pil.convert('L')  # Convert to grayscale
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485], std=[0.229]),  # Normalize using ImageNet mean and std
    ])
    frame_tensor = transform(frame_pil)
    return frame_tensor

In [ ]:
def preprocess_spectrogram(image_path):
    img = Image.open(image_path).convert('L')  # Convert to grayscale
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to match ViT input size
        transforms.ToTensor(),           # Convert to tensor
    ])
    img_tensor = transform(img)
    return img_tensor

In [ ]:
def load_dataset(videos_folder):
    X = []
    y = []
    skipped_files = []
    video_files = [file for file in sorted(os.listdir(videos_folder)) if file.endswith(".flv")]
    for video_file in tqdm(video_files):
        video_path = os.path.join(videos_folder, video_file)
        frame = extract_frame(video_path)
        if frame is not None:
            cropped_face = detect_face(frame)
            if cropped_face is not None:
                preprocessed_face = preprocess_image(cropped_face)
                X.append(preprocessed_face)
                label = video_file.split("_")[2].split(".")[0]  # Adjusted to handle different file extensions
                if label == "HAP":
                    y.append(0)
                elif label == "SAD":
                    y.append(1)
                elif label == "ANG":
                    y.append(2)
                elif label == "DIS":
                    y.append(3)
                elif label == "FEA":
                    y.append(4)
                elif label == "NEU":
                    y.append(5)
            else:
                print(f"No face detected in {video_file}. Skipping.")
                skipped_files.append(video_file[:-3])
        else:
            print(f"Failed to extract frame from {video_file}. Skipping.")
            skipped_files.append(video_file[:-3])
    return X, y, skipped_files

In [ ]:
def load_spectrogram_dataset(spectrograms_folder, skipped_files):
    X = []
    y = []
    # List all files in the input folder
    files = sorted(os.listdir(spectrograms_folder))
    # Iterate over files in the folder
    for filename in tqdm(files):
        if filename.endswith(".png") and filename[:-3] not in skipped_files:  # Assuming mel spectrograms are stored as PNG files
            input_path = os.path.join(spectrograms_folder, filename)
            img_tensor = preprocess_spectrogram(input_path)
            X.append(img_tensor)
            # Extract label from filename (assuming filename is in format "abc_IEO_label_xyz.png")
            label = filename.split("_")[2]
            if label == "HAP":
                y.append(0)
            elif label == "SAD":
                y.append(1)
            elif label == "ANG":
                y.append(2)
            elif label == "DIS":
                y.append(3)
            elif label == "FEA":
                y.append(4)
            elif label == "NEU":
                y.append(5)
    return X, y

In [ ]:
# Define the ConcatDataset class to concatenate video frame and spectrogram tensors
class ConcatDataset(torch.utils.data.Dataset):
    def __init__(self, X1, X2, y, modality='multimodal', presaved=False):
        self.X1 = X1
        self.X2 = X2
        self.y = y
        self.modality = modality
        self.presaved = presaved
    def __len__(self):
        if self.modality == 'audio':
          return len(self.X2)
        if self.modality == 'visual':
          return len(self.X1)
        return len(self.y)

    def __getitem__(self, idx):
        if not self.presaved:
          img1 = self.X1[idx]
          img2 = self.X2[idx]
          label = self.y[idx]
        else:
          img1 = torch.from_numpy(self.X1[idx]).float()  # Convert numpy array to torch tensor
          img2 = torch.from_numpy(self.X2[idx]).float()  # Convert numpy array to torch tensor
          label = torch.tensor(self.y[idx])  # Convert numpy array to torch tensor

        concatenated_img = torch.cat((img1, img2), dim=0)  # Concatenate along 0 dimension
        if self.modality == 'visual':
          return img1, label
        if self.modality == 'audio':
          return img2, label
        return concatenated_img, label # concatenate modalities

#### Train Model

In [ ]:
def train_model(model, criterion, optimizer, train_loader, device):
    model.train()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    for inputs, labels in tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    accuracy = correct_preds / total_preds
    return epoch_loss, accuracy

#### Test Model

In [ ]:
def test_model(model, criterion, test_loader, device):
    model.eval()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_preds += (predicted == labels).sum().item()
            total_preds += labels.size(0)
    epoch_loss = running_loss / len(test_loader.dataset)
    accuracy = correct_preds / total_preds
    return epoch_loss, accuracy

#### ViT Definition

In [ ]:
class ViT(nn.Module):
    # ViT architecture adapted from here - https://theaisummer.com/vision-transformer/
    def __init__(self, *,
                 img_dim,
                 in_channels=3,
                 patch_dim=16,
                 num_classes=6, # full-scale CREMA-D
                 dim=512,
                 blocks=6,
                 heads=4,
                 dim_linear_block=1024,
                 dim_head=None,
                 dropout=0, transformer=None, classification=True):
        """
        Args:
            img_dim: the spatial image size
            in_channels: number of img channels
            patch_dim: desired patch dim
            num_classes: classification task classes
            dim: the linear layer's dim to project the patches for MHSA
            blocks: number of transformer blocks
            heads: number of heads
            dim_linear_block: inner dim of the transformer linear block
            dim_head: dim head in case you want to define it. defaults to dim/heads
            dropout: for pos emb and transformer
            transformer: in case you want to provide another transformer implementation
            classification: creates an extra CLS token
        """
        super().__init__()
        assert img_dim % patch_dim == 0, f'patch size {patch_dim} not divisible'
        self.p = patch_dim
        self.classification = classification
        tokens = (img_dim // patch_dim) ** 2
        self.token_dim = in_channels * (patch_dim ** 2)
        self.dim = dim
        self.dim_head = (int(dim / heads)) if dim_head is None else dim_head
        self.project_patches = nn.Linear(self.token_dim, dim)

        self.emb_dropout = nn.Dropout(dropout)
        if self.classification:
            self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
            self.pos_emb1D = nn.Parameter(torch.randn(tokens + 1, dim))
            self.mlp_head = nn.Linear(dim, num_classes)
        else:
            self.pos_emb1D = nn.Parameter(torch.randn(tokens, dim))

        if transformer is None:
            self.transformer = TransformerEncoder(dim, blocks=blocks, heads=heads,
                                                  dim_head=self.dim_head,
                                                  dim_linear_block=dim_linear_block,
                                                  dropout=dropout)
        else:
            self.transformer = transformer

    def expand_cls_to_batch(self, batch):
        """
        Args:
            batch: batch size
        Returns: cls token expanded to the batch size
        """
        return self.cls_token.expand([batch, -1, -1])

    def forward(self, img, mask=None):
        batch_size = img.shape[0]
        img_patches = rearrange(
            img, 'b c (patch_x x) (patch_y y) -> b (x y) (patch_x patch_y c)',
                                patch_x=self.p, patch_y=self.p)
        # project patches with linear layer + add pos emb
        img_patches = self.project_patches(img_patches)

        if self.classification:
            img_patches = torch.cat(
                (self.expand_cls_to_batch(batch_size), img_patches), dim=1)

        patch_embeddings = self.emb_dropout(img_patches + self.pos_emb1D)

        # feed patch_embeddings and output of transformer. shape: [batch, tokens, dim]
        y = self.transformer(patch_embeddings, mask)

        if self.classification:
            # we index only the cls token for classification.
            return self.mlp_head(y[:, 0, :])
        else:
            return y

#### Check that the videos and spectrograms are in one-to-one correspondence

In [ ]:
assert([file[:-3] for file in sorted(os.listdir(videos_folder))] == [file[:-3] for file in sorted(os.listdir(spectrograms_folder))])

#### Load Data

In [55]:
_fullscale = True # Run fullscale experiment?
_presaved = False # Use presaved data .npy files?

# Check if data folders exists
if not os.path.exists(videos_folder):
    print("Videos folder does not exist.")
    sys.exit(1)
if not os.path.exists(spectrograms_folder):
    print("Spectrograms folder does not exist.")
    sys.exit(1)

# Load dataset and split into train and test sets
if _presaved:
  X = np.load('/content/drive/MyDrive/csci535/models/X_7439.npy', mmap_mode='r')
  y = np.load('/content/drive/MyDrive/csci535/models/y_7439.npy', mmap_mode='r')
  X_spec = np.load('/content/drive/MyDrive/csci535/models/X_spec_7439.npy', mmap_mode='r')
  y_spec = np.load('/content/drive/MyDrive/csci535/models/y_spec_7439.npy', mmap_mode='r')
else:
  X, y, skipped_files = load_dataset(videos_folder)
  X_spec, y_spec = load_spectrogram_dataset(spectrograms_folder, skipped_files)

 35%|███▌      | 2638/7439 [07:07<09:58,  8.02it/s]

No face detected in 1033_ITH_HAP_XX.flv. Skipping.


 36%|███▌      | 2662/7439 [07:11<09:49,  8.10it/s]

No face detected in 1033_MTI_HAP_XX.flv. Skipping.


 36%|███▌      | 2679/7439 [07:13<09:34,  8.28it/s]

...

 87%|████████▋ | 6478/7439 [17:34<03:33,  4.50it/s]

No face detected in 1080_IOM_ANG_XX.flv. Skipping.


 88%|████████▊ | 6530/7439 [17:45<02:20,  6.46it/s]

No face detected in 1080_TSI_HAP_XX.flv. Skipping.


100%|██████████| 7439/7439 [01:48<00:00, 68.40it/s]


#### Save data as .npy files (if needed)

In [56]:
# Save X, y, X_spec, y_spec
np.save('X_7439.npy', np.array(X))
np.save('y_7439.npy', np.array(y))
np.save('X_spec_7439.npy', np.array(X_spec))
np.save('y_spec_7439.npy', np.array(y_spec))
!cp 'X_7439.npy' '/content/drive/MyDrive/csci535/models/'
!cp 'y_7439.npy' '/content/drive/MyDrive/csci535/models/'
!cp 'X_spec_7439.npy' '/content/drive/MyDrive/csci535/models/'
!cp 'y_spec_7439.npy' '/content/drive/MyDrive/csci535/models/'

#### Split Data into Train and Test Sets

In [57]:
# Split the data into train and test sets
print(f"Total number of samples: {len(X)}")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(f"Number of train samples (video): {len(X_train)}", f"Number of test samples: {len(X_test)}")
X_train_spec, X_test_spec, y_train_spec, y_test_spec = train_test_split(X_spec, y_spec, test_size=0.3, random_state=42)
print(f"Number of train samples (audio): {len(X_train_spec)}", f"Number of test samples: {len(X_test_spec)}")

Total number of samples: 7429
Number of train samples (video): 5200 Number of test samples: 2229
Number of train samples (audio): 5200 Number of test samples: 2229


#### Check that the number of frames and spectrograms are equal and that the train + test samples add up to total samples

In [59]:
assert(len(X_train) == len(X_train_spec) and len(X_test) == len(X_test_spec))

#### Train Loop

In [60]:
def train_ViT(_modality):
  # Adjust input channels as per modality
  if _modality == 'multimodal':
    _input_channels = 2
  else:
    _input_channels = 1

  # Initialize the ViT model
  model = ViT(img_dim=224,  # Image dimension
              in_channels=_input_channels,  # Number of input channels
              patch_dim=16,  # Patch dimension
              num_classes=6,  # 6 classes for HAPPY, SAD, ANGRY, DISGUST, FEAR, NEUTRAL
              dim=768,  # Dimensionality of the token embeddings
              blocks=6,  # Number of transformer blocks
              heads=4,  # Number of attention heads
              dim_linear_block=1024,  # Dimensionality of the linear block
              dropout=0.4,  # Dropout rate
              classification=True)  # Whether or not to include a classification token

  # Define device
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)

  # Define loss function and optimizer
  _lr = 0.0001
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=_lr)

  # Concatenate datasets if multimodal
  train_dataset = ConcatDataset(X_train, X_train_spec, y_train, modality = _modality, presaved = _presaved)
  test_dataset = ConcatDataset(X_test, X_test_spec, y_test, modality = _modality, presaved = _presaved)

  # Create data loaders
  _bs = 16

  # Create data loaders
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=_bs, shuffle=True)
  test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=_bs)

  print(f"\n\nBatch size: {_bs}", f"lr: {_lr}")

  # Training loop
  num_epochs = 50
  print(f"Training ViT for \"{_modality}\" pipeline ...\n------------------------------------------------\n")
  for epoch in range(num_epochs):
      print("Epoch " + str(epoch))
      train_loss, train_accuracy = train_model(model, criterion, optimizer, train_loader, device)
      test_loss, test_accuracy = test_model(model, criterion, test_loader, device)
      print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

  # Save the model
  if _modality == 'multimodal':
    torch.save(model.state_dict(), 'ViT_audio_video_fullscale_'+str(num_epochs)+'_'+str(_bs)+'_'+str(_lr))
  elif _modality == 'audio':
    torch.save(model.state_dict(), 'ViT_audio_fullscale_'+str(num_epochs)+'_'+str(_bs)+'_'+str(_lr))
  elif _modality == 'visual':
    torch.save(model.state_dict(), 'ViT_video_fullscale_'+str(num_epochs)+'_'+str(_bs)+'_'+str(_lr))
  else:
    print("Improper modality provided!")

  return train_loss, train_accuracy, test_loss, test_accuracy

#### Choose modalities

In [61]:
# Define modalities
_modality = ['visual', 'audio', 'multimodal']

#### Train ViT

In [ ]:
# Train ViT
scores = {}
for _m in _modality:
  train_loss, train_accuracy, test_loss, test_accuracy = train_ViT(_m)
  scores[_m] = [train_loss, train_accuracy, test_loss, test_accuracy]



Batch size: 16 lr: 0.0001
Training ViT for "audio" pipeline ...
------------------------------------------------

Epoch 0


100%|██████████| 140/140 [00:07<00:00, 19.01it/s]


Epoch 1/50, Train Loss: 1.6541, Train Accuracy: 0.3023, Test Loss: 1.5680, Test Accuracy: 0.3540
Epoch 1


100%|██████████| 140/140 [00:07<00:00, 17.99it/s]


Epoch 2/50, Train Loss: 1.5593, Train Accuracy: 0.3504, Test Loss: 1.4906, Test Accuracy: 0.3952
Epoch 2


...

100%|██████████| 140/140 [00:07<00:00, 17.54it/s]


Epoch 37/50, Train Loss: 1.3737, Train Accuracy: 0.4529, Test Loss: 1.4203, Test Accuracy: 0.4424
Epoch 37


100%|██████████| 140/140 [00:07<00:00, 17.70it/s]


Epoch 38/50, Train Loss: 1.3730, Train Accuracy: 0.4538, Test Loss: 1.4306, Test Accuracy: 0.4468
Epoch 38


 62%|██████▏   | 202/325 [00:32<00:19,  6.17it/s]

In [ ]:
# Print results
print("\nResults\n---------------\n")
for _m, val in scores.items():
  print(f"Modality: {_m}, Train Loss: {val[0]:.4f}, Train Accuracy: {val[1]:.4f}, Test Loss: {val[2]:.4f}, Test Accuracy: {val[3]:.4f}")

In [ ]:
# Copy trained models to GDrive
# !cp 'ViT_audio_video_fullscale_50_16_0.0001' '/content/drive/MyDrive/csci535/models/ViT'
# !cp 'ViT_audio_fullscale_50_16_0.0001' '/content/drive/MyDrive/csci535/models/ViT'
# !cp 'ViT_video_fullscale_50_16_0.0001' '/content/drive/MyDrive/csci535/models/ViT'

In [ ]:
# Clear memory
# import gc
# gc.collect()

In [ ]:
# ! cd /content/drive/MyDrive/csci535/